---
title: "Naïve Bayes"
format: html
---

<b>*Disclaimer - this section is a work in progress</b>
<br></br>


# Introduction to Naive Bayes

Naive Bayes, a widely acclaimed machine learning algorithm, harnesses Bayes' Theorem to categorize data into predefined classes or categories. Praised for its simplicity, swift training capabilities, and robust performance, it stands as a foundational tool in data science. At its core, Bayes' Theorem calculates the probability of event A given the occurrence of event B, expressed as:
$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$
Naive Bayes accomplishes classifications by leveraging feature vectors and the principles of Bayes' Theorem to assess values. The 'naive' label in its name stems from its  assumption of independence among predictors, simplifying computational tasks. This algorithm shines in contexts featuring text and categorical data, excelling in applications like spam email identification, sentiment analysis, and document categorization. Despite its seemingly 'naive' premise, Naive Bayes consistently delivers impressive real-world performance, making it a crucial tool for various data science classification tasks.
<br></br>
Common varients of Naive Bayes include Multinomial, Guassian, and Bernoulli Naive Bayes. Multinomial Naive Bayes is the most common variant as is often used for text classification. Gaussian Naive Bayes is appropriate for continuous numerical data while Bernoulli Naive Bayes is a derivation of Multinomial Naive Bayes that is appropriate for binary or boolean data.
<br></br>
The purpose of this page is to implement Naïve Bayes classification on a variety of datasets, some of which may be more for suitable than others for this method. This work is a component of my DSAN 5000 class project.

# Prepare your Data for Naive Bayes

# Feature Selection

# Naive Bayes with Labeled Record Data

I tried applying Naive Bayes to my time series data. Since it is time series data, the procedures for classifications models like Naive Bayes are not suitable for a scenario like this.

# Naive Bayes with Labeled Text Data

# Extra Joke

Are monsters good at math? Not unless you Count Dracula.

What’s the official animal of Pi day? The Pi-thon!

What do you call a number that can’t sit still? A roamin’ numeral!